In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from scipy.stats import friedmanchisquare
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
from scipy.stats import wilcoxon
from itertools import combinations
import openpyxl

In [2]:
# Define the file name and path
file_name = "Data-til-Csv.xlsx"
file_path = os.path.join(os.getcwd(), file_name)

# Load only the "Combined" sheet
df = pd.read_excel(file_path, sheet_name="Combined")


In [3]:
# Filter to keep only rows that is correct
df = df[df["Filter-Level"] == "Rigtig"]



In [4]:
#Filter to complete 8 response in a day
def filter_numbers(df, column_name):
    # Count occurrences of each number
    number_counts = df[column_name].value_counts()

    # Identify numbers that appear exactly 8 times
    valid_numbers = number_counts[number_counts == 8].index

    # Filter the DataFrame to keep only those numbers that appear 8 times
    filtered_df = df[df[column_name].isin(valid_numbers)]

    # Identify numbers that are removed (those not appearing exactly 8 times)
    removed_numbers = number_counts[number_counts != 8].index

    # Report removed numbers
    if len(removed_numbers) > 0:
        print("Removed numbers:", removed_numbers)
    else:
        print("No numbers were removed.")

    return filtered_df

In [6]:
#Remove double response
# To remove multiple columns
df = df.drop(['Odour 2', 'Pungency 2','Air quality 2','Freshness 2','Filter-Level','TubeTal'], axis=1)

In [7]:
#filter date
# Convert the 'date' column to datetime type (if it's not already)
df['Date'] = pd.to_datetime(df['Date'])

# Specify the date you want to filter
specific_date1 = '2024-10-01'
specific_date2 = '2024-10-02'
specific_date3 = '2024-10-03'


# Filter the DataFrame to keep only the specific date

df_filtered1 = df[df['Date'] == specific_date1]
df_filtered2 = df[df['Date'] == specific_date2]
df_filtered3 = df[df['Date'] == specific_date3]


In [8]:
# filtered_df = filter_numbers(df, "Number")
df_filtered1 = filter_numbers(df_filtered1,"Number")
df_filtered2 = filter_numbers(df_filtered2,"Number")
df_filtered3 = filter_numbers(df_filtered3,"Number")


Removed numbers: Index([2, 15, 35], dtype='int64', name='Number')
Removed numbers: Index([13, 29], dtype='int64', name='Number')
Removed numbers: Index([15, 4], dtype='int64', name='Number')


In [9]:
#filter tube
dfs1 = {}
dfs2 = {}
dfs3 = {}

# Loop through the values in the "Tube" column and create separate dataframes
for tube_value in range(1, 9): 
    # Filter the data based on 'Tube' value and remove 'Date' and 'Tube' columns
    dfs1[tube_value] = df_filtered1[df_filtered1['Tube'] == tube_value].drop(['Date', 'Tube'], axis=1)
    dfs2[tube_value] = df_filtered2[df_filtered2['Tube'] == tube_value].drop(['Date', 'Tube'], axis=1)
    dfs3[tube_value] = df_filtered3[df_filtered3['Tube'] == tube_value].drop(['Date', 'Tube'], axis=1)

In [11]:
#Defining each tube
#day 1 diffrent tube response
df1_tube_1 = dfs1[1]
df1_tube_2 = dfs1[2]
df1_tube_3 = dfs1[3]
df1_tube_4 = dfs1[4]
df1_tube_5 = dfs1[5]
df1_tube_6 = dfs1[6]
df1_tube_7 = dfs1[7]
df1_tube_8 = dfs1[8]

In [12]:
#day 2 diffrent tube response
df2_tube_1 = dfs2[1]
df2_tube_2 = dfs2[2]
df2_tube_3 = dfs2[3]
df2_tube_4 = dfs2[4]
df2_tube_5 = dfs2[5]
df2_tube_6 = dfs2[6]
df2_tube_7 = dfs2[7]
df2_tube_8 = dfs2[8]

In [13]:
#day 3 diffrent tube response
df3_tube_1 = dfs3[1]
df3_tube_2 = dfs3[2]
df3_tube_3 = dfs3[3]
df3_tube_4 = dfs3[4]
df3_tube_5 = dfs3[5]
df3_tube_6 = dfs3[6]
df3_tube_7 = dfs3[7]
df3_tube_8 = dfs3[8]

In [15]:
#Freidman And Willcoxon Day 1
# Create a list of DataFrames
tube_dfs1 = [df1_tube_1, df1_tube_2, df1_tube_3, df1_tube_4, df1_tube_5, df1_tube_6, df1_tube_7, df1_tube_8]

# Column names to test
columns_to_test = ["Odour", "Pungency", "Air quality", "Freshness"]


friedman_results = []
wilcoxon_results = []

# Perform Friedman Test for each column across all 8 tubes
for col in columns_to_test:
    all_tube_data = [tube[col].values for tube in tube_dfs1]
    
    # Perform the Friedman test
    statistic, p_value = stats.friedmanchisquare(*all_tube_data)
    
    # Determine significance based on p-value 
    alpha = 0.05
    significant = "Yes" if p_value < alpha else "No"
    
    # Append the result
    friedman_results.append({
        "Column": col,
        "Statistic": statistic,
        "p-value": p_value,
        "Alpha": alpha,
        "Significant": significant
    })
    
    # If Friedman test is significant
    if significant == "Yes":
        # Get all pairs of tubes for the column
        tube_pairs = combinations(range(8), 2)
        
        # Perform Wilcoxon signed-rank test 
        for i, j in tube_pairs:
            tube_i_data = tube_dfs1[i][col].values
            tube_j_data = tube_dfs1[j][col].values
            
            # Perform Wilcoxon test
            statistic, p_value = stats.wilcoxon(tube_i_data, tube_j_data)
            
            # Determine significance
            significant_wilcoxon = "Yes" if p_value < alpha else "No"
            
            # Append the Wilcoxon test result
            wilcoxon_results.append({
                "Column": col,
                "Pair": f"Tube {i+1} vs Tube {j+1}",
                "Statistic": statistic,
                "p-value": p_value,
                "Alpha": alpha,
                "Significant": significant_wilcoxon
            })

# Convert results to DataFrames
friedman_df = pd.DataFrame(friedman_results)
wilcoxon_df = pd.DataFrame(wilcoxon_results)

# Save both results to separate sheets in the same Excel file
with pd.ExcelWriter("Friedman_Wilcoxon-Tube-Day-1.xlsx") as writer:
    friedman_df.to_excel(writer, sheet_name="Friedman Test", index=False)
    wilcoxon_df.to_excel(writer, sheet_name="Wilcoxon Test", index=False)

print("Test results saved to 'Friedman_Wilcoxon-Day-1.xlsx' with separate sheets for Friedman and Wilcoxon tests.")

Test results saved to 'Friedman_Wilcoxon-Day-1.xlsx' with separate sheets for Friedman and Wilcoxon tests.


In [17]:
#Freidman And Willcoxon Day 2
# Create a list of DataFrames
tube_dfs2 = [df2_tube_1, df2_tube_2, df2_tube_3, df2_tube_4, df2_tube_5, df2_tube_6, df2_tube_7, df2_tube_8]

# Column names to test
columns_to_test = ["Odour", "Pungency", "Air quality", "Freshness"]

# Results will be stored in separate lists for Friedman and Wilcoxon tests
friedman_results = []
wilcoxon_results = []

# Perform Friedman Test for each column across all 8 tubes
for col in columns_to_test:
    # Prepare a list of data from all tubes for the current column
    all_tube_data = [tube[col].values for tube in tube_dfs2]
    
    # Perform the Friedman test
    statistic, p_value = stats.friedmanchisquare(*all_tube_data)
    
    # Determine significance based on p-value (using alpha = 0.05)
    alpha = 0.05
    significant = "Yes" if p_value < alpha else "No"
    
    # Append the result of the Friedman test
    friedman_results.append({
        "Column": col,
        "Statistic": statistic,
        "p-value": p_value,
        "Alpha": alpha,
        "Significant": significant
    })
    
    # If Friedman test is significant, perform Wilcoxon signed-rank tests for pairwise comparisons
    if significant == "Yes":
        # Get all pairs of tubes for the column
        tube_pairs = combinations(range(8), 2)
        
        # Perform Wilcoxon signed-rank test for each pair of tubes
        for i, j in tube_pairs:
            tube_i_data = tube_dfs2[i][col].values
            tube_j_data = tube_dfs2[j][col].values
            
            # Perform Wilcoxon test
            statistic, p_value = stats.wilcoxon(tube_i_data, tube_j_data)
            
            # Determine significance based on p-value
            significant_wilcoxon = "Yes" if p_value < alpha else "No"
            
            # Append the Wilcoxon test result
            wilcoxon_results.append({
                "Column": col,
                "Pair": f"Tube {i+1} vs Tube {j+1}",
                "Statistic": statistic,
                "p-value": p_value,
                "Alpha": alpha,
                "Significant": significant_wilcoxon
            })

# Convert results to DataFrames
friedman_df = pd.DataFrame(friedman_results)
wilcoxon_df = pd.DataFrame(wilcoxon_results)

# Save both results to separate sheets in the same Excel file
with pd.ExcelWriter("Friedman_Wilcoxon-Tube-Day-2.xlsx") as writer:
    friedman_df.to_excel(writer, sheet_name="Friedman Test", index=False)
    wilcoxon_df.to_excel(writer, sheet_name="Wilcoxon Test", index=False)

print("Test results saved to 'Friedman_Wilcoxon-Day-2.xlsx' with separate sheets for Friedman and Wilcoxon tests.")

Test results saved to 'Friedman_Wilcoxon-Day-2.xlsx' with separate sheets for Friedman and Wilcoxon tests.


In [18]:
#Freidman And Willcoxon Day 3
# Create a list of DataFrames
tube_dfs3 = [df3_tube_1, df3_tube_2, df3_tube_3, df3_tube_4, df3_tube_5, df3_tube_6, df3_tube_7, df3_tube_8]

# Column names to test
columns_to_test = ["Odour", "Pungency", "Air quality", "Freshness"]

# Results will be stored
friedman_results = []
wilcoxon_results = []

# Perform Friedman Test for each column across all 8 tubes
for col in columns_to_test:n
    all_tube_data = [tube[col].values for tube in tube_dfs3]
    
    # Perform the Friedman test
    statistic, p_value = stats.friedmanchisquare(*all_tube_data)
    
    # Determine significance
    alpha = 0.05
    significant = "Yes" if p_value < alpha else "No"
    
    # Append the result of the Friedman test
    friedman_results.append({
        "Column": col,
        "Statistic": statistic,
        "p-value": p_value,
        "Alpha": alpha,
        "Significant": significant
    })
    
    if significant == "Yes":
        # Get all pairs of tubes for the column
        tube_pairs = combinations(range(8), 2)
        
        # Perform Wilcoxon signed-rank test for each pair of tubes
        for i, j in tube_pairs:
            tube_i_data = tube_dfs3[i][col].values
            tube_j_data = tube_dfs3[j][col].values
            
            # Perform Wilcoxon test
            statistic, p_value = stats.wilcoxon(tube_i_data, tube_j_data)
            
            # Determine significance based on p-value
            significant_wilcoxon = "Yes" if p_value < alpha else "No"
            
            # Append the Wilcoxon test result
            wilcoxon_results.append({
                "Column": col,
                "Pair": f"Tube {i+1} vs Tube {j+1}",
                "Statistic": statistic,
                "p-value": p_value,
                "Alpha": alpha,
                "Significant": significant_wilcoxon
            })

# Convert results to DataFrames
friedman_df = pd.DataFrame(friedman_results)
wilcoxon_df = pd.DataFrame(wilcoxon_results)

# Save both results to separate sheets in the same Excel file
with pd.ExcelWriter("Friedman_Wilcoxon-Tube-Day-3.xlsx") as writer:
    friedman_df.to_excel(writer, sheet_name="Friedman Test", index=False)
    wilcoxon_df.to_excel(writer, sheet_name="Wilcoxon Test", index=False)

print("Test results saved to 'Friedman_Wilcoxon-Day-3.xlsx' with separate sheets for Friedman and Wilcoxon tests.")

Test results saved to 'Friedman_Wilcoxon-Day-3.xlsx' with separate sheets for Friedman and Wilcoxon tests.
